In [1]:
import torch
import torch.nn as nn
from vit_pytorch import ViT
from graphs import Graph, prims
import os
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
from d2lvit import *
import copy
from collections import OrderedDict

In [2]:
model = torch.load(os.path.join('..', 'models', 'finetune_3', 'digit-model.pt'))

In [3]:
train_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_4k'), m.resnet_preprocess()) 
val_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_2k'), m.resnet_preprocess()) 
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}

Train size: 4000 Val size: 2000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [4]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

1.0


In [5]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)

1.0


In [6]:
digits_model = copy.deepcopy(model)

In [7]:
model.heads = nn.Sequential(OrderedDict([('head', nn.Linear(768, 45))]))
model = model.to(device)

In [8]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[ 9.9909e-01,  4.2757e-01,  3.7896e-01, -9.6157e-02, -5.1726e-02,
          7.8455e-02, -7.6055e-01, -5.0791e-01, -8.0184e-03, -2.0449e-02,
          3.7224e-01,  7.8065e-01, -6.5166e-01,  2.5633e-01,  4.1826e-01,
         -4.6709e-01, -2.4896e-01, -8.2779e-02, -1.3522e+00, -8.2637e-01,
          5.1893e-02, -1.8373e-01, -5.9684e-01,  1.2856e+00,  1.8863e-02,
          6.8374e-01, -3.7401e-01, -5.7203e-01, -8.2099e-01,  8.7980e-01,
          1.2567e-01, -1.3223e+00, -4.5598e-01, -8.4265e-02, -5.2702e-01,
         -9.3597e-01, -2.7036e-01, -6.9300e-01,  4.3744e-01,  1.0455e-03,
          4.6440e-01,  8.5975e-01,  3.7741e-01,  9.2724e-01,  4.8989e-02]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [9]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'finetune_3'), digits_model)

Epoch 10 done, train loss: 0.0006 val acc: 1.0000
Epoch 20 done, train loss: 0.0001 val acc: 1.0000
Epoch 30 done, train loss: 0.0001 val acc: 1.0000
Epoch 40 done, train loss: 0.0000 val acc: 1.0000
Epoch 50 done, train loss: 0.0000 val acc: 1.0000
Epoch 60 done, train loss: 0.0000 val acc: 1.0000
Epoch 70 done, train loss: 0.0000 val acc: 1.0000
Epoch 80 done, train loss: 0.0000 val acc: 1.0000
Epoch 90 done, train loss: 0.0000 val acc: 1.0000
Epoch 100 done, train loss: 0.0000 val acc: 1.0000


In [10]:
model = torch.load(os.path.join('..', 'models', 'finetune_3', 'tree-model.pt'))

In [11]:
config = {'labels_key': 'tree_label'}
train_acc = m.predict(model, train_loader, device, config, digits_model)
print(train_acc)

1.0


In [12]:
val_acc = m.predict(model, val_loader, device, config, digits_model)
print(val_acc)

1.0
